In [1]:
import pandas as pd
import numpy as np

## 데이터 로드

In [2]:
df=pd.read_excel('2차클러스터완료.xlsx')
df.head(5)

,Unnamed: 0,reviewId,scores,compound,comp_score,star,doc,thumbsUpCount,at,score,doc_id,topic_no,2nd_cluster_num
0,0,a6c7a357-da19-46a2-b265-855a9788306a,"{'neg': 0.0, 'neu': 0.954, 'pos': 0.046, 'comp...",0.2382,pos,1,if i lived in a more populated area it might ...,0,2020-09-23 10:13:55,0.505659,138523,243,6
1,1,08ac1b2f-a2f1-4ab4-8353-f148a2c47d84,"{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'comp...",-0.4235,neg,1,but ultimately the whole thing is somewhat ri...,2,2020-02-01 15:02:02,0.701373,33584,370,5
2,2,96e5ca13-05f7-4c30-91f4-286066dc2a0d,"{'neg': 0.508, 'neu': 0.492, 'pos': 0.0, 'comp...",-0.4767,neg,1,fake profiles money scammers,0,2020-10-27 01:24:25,0.852542,34482,135,5
3,3,4ac73bb7-4dfe-4002-b9c6-1606ed772507,"{'neg': 0.126, 'neu': 0.874, 'pos': 0.0, 'comp...",-0.3804,neg,2,i ve been using the app for quite a while so i...,0,2021-01-12 16:47:57,0.701085,216063,41,7
4,4,f81cb35e-1777-4d00-8c0b-6e42b22937ae,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos,4,while the permission for the camera is set to ...,107,2021-06-29 07:05:08,0.509364,204907,169,10


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267352 entries, 0 to 267351
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       213665 non-null  float64
 1   reviewId         213665 non-null  object 
 2   scores           213665 non-null  object 
 3   compound         213665 non-null  float64
 4   comp_score       213665 non-null  object 
 5   star             213665 non-null  float64
 6   doc              213665 non-null  object 
 7   thumbsUpCount    213665 non-null  float64
 8   at               213665 non-null  object 
 9   score            213665 non-null  float64
 10  doc_id           213665 non-null  float64
 11  topic_no         213665 non-null  float64
 12  2nd_cluster_num  213665 non-null  float64
dtypes: float64(8), object(5)
memory usage: 26.5+ MB


### 확실한 토픽 결정
- topic1은 토픽으로 묶일 수 없는 것들이라고 판단하여 제거
- 스와이핑: topic0, topic8, topic11, topic13, topic14
- 지불: topic2, topic8, topic13
- 차단: topic5
- 메세지: topic7, topic12
- 사기: topic10
- 인종차별:topic12
- 젠더: topic12
- 외모: topic12
- 오류(bug): topic9, topic13, topic 14
- 인터렉션 오류: topic3
- 인증: topic6
- 광고: topic8
- 디자인: topic14
- 가벼운 관계: topic4
- 진지한 관계: topic4

### 애매한 토픽
- topic0, topic8, topic9, topic12, topic13, topic14


In [4]:
# 여러가지 토픽이 섞여있는 클러스터의 경우 따로 변수에 저장 (수동으로 토픽 분류를 위해)
# 토픽이 섞여있는 + 필요없는 내용과 섞여있는 클러스터의 경우 특정 단어(분석자가 데이터를 보고 직접 설정)가 들어있는 리뷰를 추출 후 특정 토픽으로 할당하는 과정을 거침
df1=df.loc[(df['2nd_cluster_num']==0)]
print(len(df1))
df2=df.loc[(df['2nd_cluster_num']==8)]
print(len(df2))
df3=df.loc[(df['2nd_cluster_num']==9)]
print(len(df3))
df4=df.loc[(df['2nd_cluster_num']==12)]
print(len(df4))
df5=df.loc[(df['2nd_cluster_num']==13)]
print(len(df5))
df6=df.loc[(df['2nd_cluster_num']==14)]
print(len(df6))
df7=df.loc[(df['2nd_cluster_num']==4)]
print(len(df7))

13281
6636
5822
10224
4646
2767
9370


### 스와이핑

In [5]:
# 확정된 토픽
# topic 11,14
df.loc[df['2nd_cluster_num']==11,'label']='스와이핑'
df.loc[df['2nd_cluster_num']==14,'label']='스와이핑'

# 애매한 토픽 
# topic0, topic8, topic13 
# 스와이핑관련 단어만 추출 (데이터 보면서 직접 결정)
swipe=['swipe', 'swiping','swiped','unmatch','bot','swipes','algorithm','pof', 'filter']

#topic0
SW='|'.join(swipe)
swiping_alforithm=df1[df1['doc'].str.contains(SW)]
print( len(swiping_alforithm))

#topic 8 
swiping_alforithm1=df2[df2['doc'].str.contains(SW)]
print(len(df2[df2['doc'].str.contains(SW)]))

#topic13 
swiping2=df5[df5['doc'].str.contains(SW)]
print(len(df5[df5['doc'].str.contains(SW)]))


# 합치기
t_13_8=pd.concat([swiping_alforithm1,swiping2])
t_13_8['label']='스와이핑'
swiping=pd.concat([t_13_8,df[df.label=='스와이핑']])

1518
838
153


### 지불

In [6]:
# 확정된 토픽
# topic 2 
df.loc[df['2nd_cluster_num']==2,'label']='지불'

# 애매한 토픽
# topic 8, 13 
# 지불 단어만 추출 (데이터 보면서 직접 결정)
pay=['subscription','subscriptions','subscribing','unsubscribed','subscriber','subscribed',
     'memberships','cash','refunded','billing', 'payed','subscribe','paywall','charges','charge',
    'paying', 'payment', 'pay', 'paid', 'fee','overpriced','monthly','unpaid','billed', 'bought',
    'primium','expired']

PA='|'.join(pay)

#swiping에서 할당되지 않은 것에서 payment에 해당되는거 할당
he=df2[~df2['doc'].str.contains(SW)]
payment=he[he['doc'].str.contains(PA)]
print(len(df2[df2['doc'].str.contains(PA)]))


#애매한 데이터셋(df5) payment1에 해당되는거 할당
he10=df5[df5['doc'].str.contains(SW)]
payment1=he10[he10['doc'].str.contains(PA)]
print(len(df5[df5['doc'].str.contains(PA)]))


# 합치기
t_8_13=pd.concat([payment,payment1])
t_8_13['label']='지불'
Payment=pd.concat([t_8_13,df[df.label=='지불']])

3462
570


### 차단

In [7]:
df.loc[df['2nd_cluster_num']==5,'label']='차단'
ban=df[df['label']=='차단']
print(len(ban))

12211


### 메세지

In [8]:
#확정된 토픽
df.loc[df['2nd_cluster_num']==7,'label']='메세지'

#애매한 토픽
#topic12
mes=['responses','respond','response','replying','responds','responding','replies','reply',
    'responded','answered','ignores','ignore','rude','answering','answers','replied',
    'ignored','ghosted','messages','chatted','ignoring']


ME='|'.join(mes)
message=df4[df4['doc'].str.contains(ME)]
print(len(df4[df4['doc'].str.contains(ME)]))


#합치기
message['label']='메세지'
Message=pd.concat([message,df[df.label=='메세지']])

417


### 사기

In [9]:
#확정된 토픽
df.loc[df['2nd_cluster_num']==10,'label']='사기'
scam=df[df.label=='사기']
print(len(scam))

20672


### 인종차별

In [10]:
#애매한 토픽
#topic12
rac=['racist','white','blak','asian','asia','ethnicity','racism','race','discriminatory','indian','diversity',
     'discrimination']

RC='|'.join(rac)
h2=df4[~df4['doc'].str.contains(ME)]
racist=h2[h2['doc'].str.contains(RC)]
print(len(df4[df4['doc'].str.contains(RC)]))

racist['label']='인종차별'

619


### 젠더

In [11]:
#애매한 토픽
#topic 12
gen=['lesbian', 'gay','queer','lgbtq','bi','lgbt','transgender','nonbinary','genders',
    'trans','binary']

GE='|'.join(gen)
h3=h2[~h2['doc'].str.contains(RC)]
gender=h3[h3['doc'].str.contains(GE)]
print(len(df4[df4['doc'].str.contains(GE)]))

gender['label']='다양한_젠더'
print(len(gender))

1097


### 외모

In [12]:
#애매한 토픽
#topic 12
appear=['obese','overweight','fat','weight','ugly','ugliest',
       'pretty','handsome','gorgeous','beautiful','hot','cute','superficial',
       'fit','slim','overweight','losers']

AP='|'.join(appear)
h4=h3[~h3['doc'].str.contains(GE)]
appearance=h4[h4['doc'].str.contains(AP)]
print(len(df4[df4['doc'].str.contains(AP)]))

appearance['label']='외모'
print(len(appearance))

2192


### 오류

In [13]:
#애매한 토픽
#topic 9, topic13, topic 14
bug=['bug','bugged','buggy','laggy','lag','lags','bugs','crashes','crashed','crash',
    'glitches','glitched','glitch','glitching','lacks','crashing','fixes','fix','fixed','error','errors',
    'loading','loaded','loads','load','slow','troubleshooting','uploading',
    'problems','issues','upload','constantly','trouble','slower','browsing', 'faster','slowly','pace','speed','quickly',
    'weak','delay','freezes','unreliable']

#topic 9
BG='|'.join(bug)
bugging=df3[df3['doc'].str.contains(BG)]
print(len(df3[df3['doc'].str.contains(BG)]))

#topic 13
he11=he10[~he10['doc'].str.contains(PA)]
bugging1=he11[he11['doc'].str.contains(BG)]
print(len(df5[df5['doc'].str.contains(BG)]))

# topic 14
bugging2=df6[df6['doc'].str.contains(BG)]
print(len(df6[df6['doc'].str.contains(BG)]))

# 합치기
t_9_13_14=pd.concat([bugging,bugging1,bugging2])
t_9_13_14['label']='오류문제'
error=pd.concat([t_9_13_14,df[df.label=='오류문제']])

1520
473
419


### 인터렉션 안정

In [14]:
df.loc[df['2nd_cluster_num']==3,'label']='인터렉션안정'

connection=df[df.label=='인터렉션안정']
print(len(connection))

6990


### 인증

In [15]:
#확정된 토픽
df.loc[df['2nd_cluster_num']==6,'label']='인증'
ver=df[df['label']=='인증']
print(len(ver))

8230


### 광고

In [16]:
#애매한 토픽
#topic 8
adv=['ads', 'monetization','advertising','advertisement','ad','advertise','advertised',
     'commercial','promoting']

AD='|'.join(adv)


he1=he[~he['doc'].str.contains(PA)]
advertisement=he1[he1['doc'].str.contains(AD)]
print(len(df2[df2['doc'].str.contains(AD)]))

advertisement['label']='광고'
print(len(advertisement))

1599


### 디자인: topic

In [17]:
#애매한 토픽
#topic 14
design=df6[~df6['doc'].str.contains(BG)]
design['label']='디자인'
print(len(design))

8090


### 가벼운 관계

In [18]:
#애매한 토픽
#topic4
easy_rel=['hookers', 'prostitute','prostitutes','prostitution','porn','hoes','hookups','sex','nudes',
         'hookup','thots','hooking','ass','dirty','drugs','nude','drug','harassed','nudity','sexual','ghosted']

ER='|'.join(easy_rel)
easy_relationship=df7[df7['doc'].str.contains(ER)]
print(len(df7[df7['doc'].str.contains(ER)]))

easy_relationship['label']='가벼운관계'
print(len(easy_relationship))

1200


### 진지한 관계

In [19]:
#애매한 토픽
#topic4
stable_rel=['girlfriend', 'gf','married','engaged','marriage','boyfriend','soulmate',
         'bf','wife','partner','husband','engagement','partners','cupid','friendships','friendship','friends','mature']

SR='|'.join(stable_rel)
heo=df7[~df7['doc'].str.contains(ER)]
stable_relationship=heo[heo['doc'].str.contains(SR)]
print(len(df7[df7['doc'].str.contains(SR)]))

stable_relationship['label']='진지한관계'

850


## 라벨링한 데이터프레임 전부 합치기

In [21]:
total= pd.concat([swiping,ver,Payment,ban, Message,scam,racist,gender,appearance, connection, error, easy_relationship,stable_relationship,advertisement,design])
total.label.value_counts()

오류문제      29211
메세지       27892
스와이핑      27540
사기        20672
차단        12211
인증         8559
지불         8313
디자인        8087
인터렉션안정     7323
외모         1931
가벼운관계      1122
다양한_젠더     1045
광고          826
진지한관계       800
인종차별        619
Name: label, dtype: int64

### 리뷰문장에서 단어 4개 이상인 것만 의미있는 문장으로 보겠다.

In [22]:
a=[]
for i in total.doc: # total.doc; 리뷰 문장 컬럼
    a.append(i.split(' '))
total['token']=a


b= []
for j in total.token:
    leng=len(j)
    if leng >4:
        b.append(1) # 의미있는 문장
    else :
        b.append(0) # 의미없는 문장
        
total['length']=b
fin=total[total['length']==1] # 의미있는 문장
fin.to_excel('토픽정의_완료.xlsx')

In [23]:
fin.label.value_counts()

메세지       26334
스와이핑      26058
오류문제      25774
사기        18193
차단        11841
인증         8230
지불         8009
인터렉션안정     6990
디자인        5901
외모         1738
가벼운관계      1003
다양한_젠더      991
광고          759
진지한관계       748
인종차별        552
Name: label, dtype: int64

# 중복데이터 확인하기
- 같은 토픽에서 여러개의 키워드를 뽑아낸 경우

In [24]:
ji=fin[(fin.label=='인종차별')|(fin.label=='외모')|(fin.label=='다양한_젠더')]
ji[ji.doc.duplicated()].label.value_counts()

Series([], Name: label, dtype: int64)

In [25]:
ji=fin[(fin.label=='광고')|(fin.label=='지불')|(fin.label=='스와이프')]
ji[ji.doc.duplicated()].label.value_counts()

Series([], Name: label, dtype: int64)

In [26]:
#전체 중복 데이터 
print(len(fin[fin.doc.duplicated()]))

0


In [27]:
fin[fin.doc.duplicated()].label.value_counts()

Series([], Name: label, dtype: int64)

## 토픽별 빈도 top10개 뽑기

In [30]:
df=pd.read_excel('토픽정의_완료.xlsx')
print(df['label.1'].unique())

['스와이핑' '인증' '지불' '차단' '메세지' '사기' '인종차별' '다양한_젠더' '외모' '인터렉션안정' '오류문제'
 '가벼운관계' '진지한관계' '광고' '디자인']


In [31]:
# 각 토픽 변수로 지정
sw=df[df['label.1']=='스와이핑']
ver=df[df['label.1']=='인증']
pa=df[df['label.1']=='지불']
bl=df[df['label.1']=='차단']
me=df[df['label.1']=='메세지']
fr=df[df['label.1']=='사기']
ra=df[df['label.1']=='인종차별']
ge=df[df['label.1']=='다양한_젠더']
er=df[df['label.1']=='오류문제']
ap=df[df['label.1']=='외모']
sta=df[df['label.1']=='인터렉션안정']
ad=df[df['label.1']=='광고']
lr=df[df['label.1']=='가벼운관계']
sr=df[df['label.1']=='진지한관계']
de=df[df['label.1']=='디자인']

In [32]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
import wordcloud
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize, pos_tag
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/heojiyeon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
# NLTK Stop words
import gensim, spacy, logging, warnings
from nltk.corpus import stopwords
import re

stop_words = stopwords.words('english')

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # 이메일 제거
        sent = re.sub('\s+', ' ', sent)  # newline chars 제거
        sent = re.sub("\'", "", sent)  # single quotes 제거
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# 리스트로 변환(data변수에 앞에 정의해둔 토픽변수이름.values.tolist() 넣어줘야함)
data = sta.doc.values.tolist() # {}.doc; 문장 들어있는 컬럼 {}에는 토픽별로 만들어진 변수 넣음
data_words = list(sent_to_words(data))
print(data_words[:1])


# bigram and trigram(combinations of two words or three words) 모델 생성
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # 터미널에서 한번만 실행
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN']): #, 'ADJ', 'VERB', 'ADV'
    """Stopwords, Form Bigrams, Trigrams and Lemmatization 제거"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp=spacy.load("en_core_web_sm")
    #nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # Stopwords, Form Bigrams, Trigrams and Lemmatization 제거
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # data_words; 앞서 전처리된 문장이 담긴 리스트

#빈리스트 제거
a=[]
for i in data_ready:
    if i==[]:
        pass
    else:
        a.append(i)

#array로 변환 for making dataframe
a1 = np.array(a)
a2=pd.DataFrame(a1)
a2.columns=['noun']

2022-12-08 01:55:00.615659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[['it', 'is', 'great', 'app', 'until', 'it', 'stops', 'working', 'and', 'the', 'great', 'person', 'was', 'talking', 'to', 'no', 'longer', 'can', 'then', 'have', 'to', 'reset', 'and', 'it', 'only', 'brings', 'up', 'old', 'matches']]


/var/folders/sf/g5ffdfm57wv22zjhcdsjt1rm0000gn/T/ipykernel_66122/1161872415.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a1 = np.array(a)


In [34]:
#빈리스트 제거
b1=[]
for i in a2['noun']:
    if i==[]:
        pass
    else:
        b1.extend(i)
a1=[]
for n in a2['noun']:
    a1.append(' '.join(n))

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

vect = CountVectorizer()
document_term_matrix = vect.fit_transform(a1) # 문서-단어 행렬 
tf = pd.DataFrame(document_term_matrix.toarray(), columns=vect.get_feature_names()) # TF (Term Frequency) 

/Users/heojiyeon/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [35]:
b1_1=tf.T.sum(axis=1)
b1_1=pd.DataFrame(b1_1)
mi=min(b1_1[0])
ma=max(b1_1[0])

k1=[]
v1=[]
for k, v in zip(b1_1.index,b1_1[0]):
    n=round(int(v-mi)/int(ma-mi),3)
    k1.append(k)
    v1.append(n)
    
dictionary_1 = dict(zip(k1, v1))
d_1={k: v for k, v in sorted(dictionary_1.items(),reverse=True, key=lambda item: item[1])}
counts_4 = Counter(d_1)

In [36]:
counts_4

Counter({'account': 1.0,
         'app': 0.595,
         'message': 0.534,
         'time': 0.383,
         'log': 0.275,
         'people': 0.237,
         'phone': 0.202,
         'login': 0.194,
         'button': 0.191,
         'match': 0.179,
         'profile': 0.176,
         'day': 0.17,
         'number': 0.132,
         'error': 0.119,
         'money': 0.119,
         'password': 0.112,
         'way': 0.105,
         'screen': 0.096,
         'sign': 0.089,
         'change': 0.088,
         'problem': 0.084,
         'issue': 0.082,
         'email': 0.075,
         'reason': 0.073,
         'notification': 0.07,
         'week': 0.07,
         'month': 0.068,
         'work': 0.067,
         'option': 0.066,
         'code': 0.064,
         'conversation': 0.064,
         'person': 0.063,
         'thing': 0.063,
         'page': 0.062,
         'scroll': 0.061,
         'hour': 0.06,
         'chat': 0.051,
         'load': 0.049,
         'access': 0.048,
         'use